<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Capstone Project

https://capstone-project.streamlit.app/

## Contents:
- [Problem Statement](#Problem-Statement)
- [About Retrieval Augmented Generation (RAG) and Fine-tuning](#About-Retrieval-Augmented-Generation-(RAG)-and-Fine-tuning)
- [RAG](#RAG)
- [Fine-tuning](#Fine-tuning)
- [Evaluation](#Evaluation)

## Problem Statement
Place your problem statement here

## About Retrieval Augmented Generation (RAG) and Fine-tuning

In this project, I used the architecture that powers ChatGPT, a generative AI tool that has revolutionized the way users get answers to their questions. To build a custom chatbot using ChatGPT's Large Language Model (LLM), two techniques, Retriever Augmented Generation (RAG) and model Fine-tuning is used.

RAG adjusts knowledge the LLM has access to through external knowledge retrieval, while fine-tuning adjusts the behaviour of the LLM for specific domains by training it on specific dataset.

## Import libraries, API

In [3]:
!pip install datasets==2.14.6
!pip install langchain==0.0.319
!pip install llama-index==0.8.48
!pip install ragas==0.0.19
!pip install openai==0.28.1

  Using cached openai-0.28.1-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.14.3
    Uninstalling openai-1.14.3:
      Successfully uninstalled openai-1.14.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.0.6 requires openai<2.0.0,>=1.10.0, but you have openai 0.28.1 which is incompatible.


In [15]:
from llama_index import Document, GPTVectorStoreIndex, ServiceContext
from llama_index import SimpleDirectoryReader # For PDF and HTML
from llama_index import download_loader # For CSV
from llama_index.llms import OpenAI
from llama_index.evaluation import DatasetGenerator
from pathlib import Path

import os
import openai

In [4]:
# Set OpenAI API key from the environment variable
from google.colab import userdata
openai.api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai.api_key

## [Optional] Create a separate csv file for each entity

In [ ]:
'''
import csv
import os

def split_rows_by_column(input_file, output_folder, encoding='utf-8'):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Read the input CSV file
    with open(input_file, 'r', newline='', encoding=encoding) as csvfile:
        reader = csv.reader(csvfile)

        # Skip the header row if it exists
        header = next(reader, None)

        # Dictionary to store file writers for each unique value in Column 1
        file_writers = {}

        # Iterate through rows and write them to separate CSV files
        for row in reader:
            key = row[0]  # Assuming Column 1 is at index 0
            if key not in file_writers:
                # Create a new CSV file for each unique value in Column 1
                file_path = os.path.join(output_folder, f'{key}_output.csv')
                file_writers[key] = csv.writer(open(file_path, 'w', newline='', encoding=encoding))

                # Write the header to the new CSV file
                if header:
                    file_writers[key].writerow(header)

            # Write the current row to the appropriate CSV file
            file_writers[key].writerow(row)

# Example usage
input_csv_file = 'data/authentic-restaurants-reviews.csv'
output_folder = 'data'

split_rows_by_column(input_csv_file, output_folder)
'''

## RAG

### Load the data

[LlamaIndex's documentation](https://gpt-index.readthedocs.io/en/latest/index.html)


LlamaIndex loads data via data connectors. The easiest and most commonly used data connector is the `SimpleDirectoryReader`, which creates documents out of every file in the given directory, 'webpages' in this case. 'webpages' consists of data of various entities saved from their respective websites in **PDF and html format**.
<br>A `Document` is a collection of text data and metadata about that data.

In [ ]:
#Use this code if your data set is in PDF / HTML format
'''
filename_fn = lambda filename: {'file_name': filename}
reader = SimpleDirectoryReader('data', exclude_hidden=True, file_metadata=filename_fn)
docs = reader.load_data()
print([x.doc_id for x in docs])
print(f"Loaded {len(docs)} docs")
'''

In [16]:
#Use this code if your data set is in CSV format
PagedCSVReader = download_loader("PagedCSVReader")

loader = PagedCSVReader(encoding="utf-8")
docs = loader.load_data(file=Path('/content/drive/MyDrive/Colab Notebooks/openai/data/whisky-reviews.csv'))

print([x.doc_id for x in docs])
print(f"Loaded {len(docs)} docs")

['c03af925-da8f-4fa0-97a1-e9c49ca99344', '06701d59-a18b-4de1-87a2-74ffd4ee1a97', '215eac35-ec6d-4b1f-b126-1b41bcf40634', 'e4cf3496-f85e-4024-9c76-95eac46232a1', 'a3cc859c-2625-4f5e-b439-b0c08254e43b', 'b88d25d3-775d-43df-af5a-97f1e826c4bb', 'ca9cd3e2-48dd-4a42-a451-47bc517f0276', '7c2fc7ce-7366-495c-a496-7aae7cd6b99b', '743c9bd6-fbe9-4db1-ba27-5e6373dfe881', '09e0be6a-9e13-4c72-8645-b6c727c91bd1', '97cfa89d-438a-47de-9451-d5e110dc80f4', '6e814889-c41e-4914-ae53-0e7d82ef6df3', '3fdf4366-f541-470a-b148-c8c38bccac71', '501fa31e-c44f-4f37-8afb-402d7af632cd', '0fb01756-8097-4800-b77c-647d15256943', '3d180488-fa9f-4f71-9f3f-7a72b340f529', '5efcc4fb-3815-4352-950c-c0b050861e41', '9638c7f9-a1df-459f-b642-b41dbd74fadd', '7069c463-6c2c-4a2a-8943-738e51f3492b', '62b921fd-b8a4-4edd-9acc-5c9f642bf521', 'c82d13ed-7d5e-42c8-837e-19870fa011c9', '37b77dd9-bbc7-44b7-befb-a4a799d2d86e', '3546596d-6eba-4274-832b-cd3e3d001fa7', 'f1f93ae8-944b-4052-bc0f-962537b5609a', '37a51d72-4f3f-48ae-bbe0-b05c7a368958',

### Indexing and Storage

With all the data loaded, a list of [N] Document objects is created. Proceed to build an Index over these objects, which makes it ready for querying by an LLM. There are 4 types of indexing: Summary index, VectorStore Index, Tree Index and Keyword Table Index.

In this project, `VectorStoreIndex` is used as it is by far the most frequent type of indexing. It takes the Documents and splits them up into Nodes, then creates `vector embeddings` of the text of every node. `Vector embedding` aka embedding is a numerical representation of the semantics, or meaning of the text. Two pieces of text with similar meanings will have mathematically similar embeddings, even if the actual text is quite different. This mathematical relationship enables semantic search, where a user provides query terms and LlamaIndex can locate text that is related to the meaning of the query terms rather than simple keyword matching. This is a big part of how Retrieval-Augmented Generation works.

Definition of classes:
- `ServiceContext` is a bundle of configuration data which can be passed to other stages of the pipeline.

In [ ]:
# Instantiate the LLM (gpt-3.5-turbo) from OpenAI and pass it to ServiceContext().
# The GPTVectorStoreIndex will use gpt-3.5-turbo as embedding model to index the documents
service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo", temperature=0)) # degree of randomness from 0 to 1.
index = GPTVectorStoreIndex.from_documents(documents=docs, service_context=service_context)

C:\Users\kisha\AppData\Local\Temp\ipykernel_6748\3671114014.py:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo", temperature=0)) # degree of randomness from 0 to 1.


After indexing, the ouput is stored in disk using the built-in .persist() method to avoid the time and cost of having to re-index it.

In [ ]:
index.storage_context.persist(persist_dir="data/index.vecstore")

## Fine-tuning

Currently, there are three key integrations with [LlamaIndex for fine-tuning](https://gpt-index.readthedocs.io/en/latest/optimizing/fine-tuning/fine-tuning.html).

In this project, since GPT-3.5-Turbo is used, I will try to distill a better model (e.g. GPT-4) into the simpler/cheaper model (e.g. GPT-3.5), i.e. finetuning GPT-3.5-turbo to ouput GPT-4 responses.

The key steps are:
1) Split the documents into train/evaluation set
1) Generate a question/answer dataset over the train set
    - use GPT-3.5-Turbo to generate questions from the external data, and GPT-4 query engine to generate answers.
    - `OpenAIFineTuningHandler` callback automatically logs questions/answers to a dataset.
2) Launch a finetuning job with `OpenAIFinetuneEngine`, and get back a finetuned model
3) Evaluate the performance of the finetuned model and compare with the base model


### Generate Train Dataset

This dataset is for finetuning the base model.

In [ ]:
# Shuffle the documents
import random

random.seed(42)
random.shuffle(docs)

gpt_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0)
)

C:\Users\kisha\AppData\Local\Temp\ipykernel_6748\3030904786.py:7: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  gpt_context = ServiceContext.from_defaults(


In [ ]:
# To avoid RuntimeError: asyncio.run() cannot be called from a running event loop
# The below code is to unblock: nest the event loops

import nest_asyncio
nest_asyncio.apply()

In [ ]:
question_gen_query = (
    "You are a whisky recommender. You are providing recommendation to new whisky lovers. \
    Your task is to recommend whisky to users based on their smell, taste and finish preferences. \
    Using the provided context from documents on different whiskys' nose, taste, finish and score, \
    formulate a single question that captures an important fact from the context. \
    Restrict the question to the context information provided."
)

# find out more about question generation from
# https://gpt-index.readthedocs.io/en/latest/examples/evaluation/QuestionGeneration.html

dataset_generator = DatasetGenerator.from_documents(
    docs[:50],
    question_gen_query=question_gen_query,
    service_context=gpt_context,
)

C:\Users\kisha\AppData\Roaming\Python\Python311\site-packages\llama_index\core\evaluation\dataset_generation.py:212: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


In [ ]:
questions = dataset_generator.generate_questions_from_nodes(num=40)
print("Generated ", len(questions), " questions")

Generated  40  questions


C:\Users\kisha\AppData\Roaming\Python\Python311\site-packages\llama_index\core\evaluation\dataset_generation.py:309: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [ ]:
with open("train_questions.txt", "w") as f:
    for question in questions:
        f.write(question + "\n")

![train_questions.png](train_questions.png)

### Generate Evaluation Dataset

This dataset is for subsequent evaluation step to measure the performance of the models.
<br> Questions are generated from a different set of documents.

In [ ]:
dataset_generator = DatasetGenerator.from_documents(
    docs[
        51:100
    ],  # since we generated question for the first 40 documents, we can skip the first 40
    question_gen_query=question_gen_query,
    service_context=gpt_context,
)

C:\Users\kisha\AppData\Roaming\Python\Python311\site-packages\llama_index\core\evaluation\dataset_generation.py:212: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


In [ ]:
questions = dataset_generator.generate_questions_from_nodes(num=20)
print("Generated ", len(questions), " questions")

Generated  20  questions


C:\Users\kisha\AppData\Roaming\Python\Python311\site-packages\llama_index\core\evaluation\dataset_generation.py:309: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [ ]:
with open("eval_questions.txt", "w") as f:
    for question in questions:
        f.write(question + "\n")

![Screenshot of eval generated](eval_questions.png)

### GPT-4 to Generate Training Data

In [39]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager

finetuning_handler = OpenAIFineTuningHandler()
callback_manager = CallbackManager([finetuning_handler])

gpt_4_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo-0613", temperature=0),
    context_window=2048,  # limit the context window artifically to test refine process
    callback_manager=callback_manager,
)

In [40]:
questions = []
with open("/content/drive/MyDrive/Colab Notebooks/openai/train_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [41]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(docs, service_context=gpt_4_context)

query_engine = index.as_query_engine(similarity_top_k=2)

In [42]:
for question in questions:
    response = query_engine.query(question)

### Create `OpenAIFinetuneEngine`

`OpenAIFinetuneEngine` is a finetune engine that will take care of launching a finetuning job, and returning an LLM model that can be directly plugged in to the rest of LlamaIndex workflows.

In [43]:
finetuning_handler.save_finetuning_events("finetuning_events.jsonl")

Wrote 40 examples to finetuning_events.jsonl


In [27]:
from llama_index.finetuning import OpenAIFinetuneEngine

finetune_engine = OpenAIFinetuneEngine(
    "gpt-3.5-turbo-0613",
    "finetuning_events.jsonl"
)

In [45]:
finetune_engine.finetune()

Num examples: 40
First example:
{'role': 'system', 'content': "You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."}
{'role': 'user', 'content': 'Context information is below.\n---------------------\nSmell: Yep, that is a peat monster all right. Huge lumbering peat, tar, lemon, caramel and light wine funk, ashy char, funky new make spirit, spice, quite hot\nTaste: Thick mean peat, huge and oily on the palate, vanilla custard, lemon, funky and sweet new make spirit, caramel, gritty char and spice. It\'s very hot, has some funky youthful notes, but it\'s not a mess and there\'s some nice flavor.\nFinish: Huge powerful peat and lots of peppery spice, bright lemon, tar, vanilla\nScore: 6\nW

![notification of successful finetuned job](finetune_job.png)

In [28]:
finetune_engine.get_current_job()

ValueError: Must call finetune() first

In [25]:
ft_llm = finetune_engine.get_finetuned_model(temperature=0)

NameError: name 'finetune_engine' is not defined

## Evaluation

To measure the performance of the pipeline, whether it is able to generate relevant and accurate responses given the external data source and a set of queries, we use 2 evaluation metrics from [`ragas` evaluation library](https://github.com/explodinggradients/ragas/tree/main/docs/concepts/metrics). Ragas uses LLMs under the hood to compute the evaluations.

The performance of the base model, gpt-3.5-turbo, will be compared with the fine-tuned model.

Computation of evaluation metrics require 3 components:
1) `Question`: A list of questions that could be asked about my external data/documents, generated using .generate_questions_from_nodes in above fine-tuning step<br>
2) `Context`: Retrieved contexts corresponding to each question. The context represents (chunks of) documents that are relevant to the question, i.e. the source from where the answer will be generated.<br>
3) `Answer`: Answer generated corresponding to each question from baseline and fine-tuned model.

The two metrics are as follow:

- `answer_relevancy` - Measures how relevant the generated answer is to the question, where an answer is considered relevant when it <u>directly</u> and <u>appropriately</u> addresses the orginal question, i.e. answers that are complete and do not include unnecessary or duplicated information. The metric does not consider factuality. It is computed using `question` and `answer`, with score ranging between 0 and 1, the higher the score, the better the performance in terms of providing relevant answers. To calculate this score, the LLM is prompted to generate an appropriate question for the generated answer multiple times, and the mean cosine similarity between these generated questions and the original question is measured. The underlying idea is that if the generated answer accurately addresses the initial question, the LLM should be able to generate questions from the answer that align with the original question, i.e. high mean cosine similarity, translating to high score.


- `faithfulness` - Measures how factually accurate is the generated answer, i.e. if the response was hallucinated, or based on factuality (from the context). It is computed from `answer` and `context`, with score ranging between 0 and 1, the higher the score, the better the performance in terms of providing contextually accurate information. To calculate this score, the LLM identifies statements within the generated answer and verifies if each statement is supported by the retrieved context. The process then counts the number of statements within the generated answer that can be logically inferred from the context, and dvide by the total number of statements in the answer.

Additional note: Cosine similarity is a metric used to measure how similar two items are. Mathematically, it measures the cosine of the angle between two vectors projected in a multi-dimensional space. The output value ranges from 0–1 where 0 means no similarity, whereas 1 means that both the items are 100% similar.
<br>Hallucinations refer to instances where the language model produces information or claims that are not accurate or supported by the input context.

Resources:
<br>https://cobusgreyling.medium.com/rag-evaluation-9813a931b3d4
<br>https://blog.langchain.dev/evaluating-rag-pipelines-with-ragas-langsmith/
<br>https://medium.aiplanet.com/evaluate-rag-pipeline-using-ragas-fbdd8dd466c1

### Evaluation of base model: GPT-3.5-Turbo

In [19]:
questions = []
with open("/content/drive/MyDrive/Colab Notebooks/openai/eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [17]:
from llama_index import VectorStoreIndex

# limit the context window to 2048 tokens so that refine is used
gpt_context = ServiceContext.from_defaults(
    # If finetuning on openai website, replace the model name accordingly
    llm=OpenAI(model="ft:gpt-3.5-turbo-0613:personal::98TuYHTg", temperature=0), context_window=2048

    # If finetuning on localhost, uncomment this code
    # llm=OpenAI(model="gpt-3.5-turbo", temperature=0), context_window=2048
)

index = VectorStoreIndex.from_documents(docs, service_context=gpt_context)

# as_query_engine builds a default retriever and query engine on top of the index
# We configure the retriever to return the top 2 most similar documents, which is also the default setting
query_engine = index.as_query_engine(similarity_top_k=2)

In [20]:
contexts = []
answers = []

for question in questions:
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))

In [21]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness

ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

result = evaluate(ds, [faithfulness, answer_relevancy])
print(result)

evaluating with [faithfulness]


100%|██████████| 2/2 [02:56<00:00, 88.18s/it]
/usr/local/lib/python3.10/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.10/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [answer_relevancy]


100%|██████████| 2/2 [00:30<00:00, 15.23s/it]
/usr/local/lib/python3.10/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.10/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


{'ragas_score': 0.8468, 'faithfulness': 0.7486, 'answer_relevancy': 0.9746}


In [ ]:
df_gpt_35 = result.to_pandas()

# Export cleaned dataframe as .csv
df_gpt_35.to_csv("eval/df_gpt_35.csv",index=False)

### Evaluation of fine-tuned model

Run the fine-tuned model on the evaluation dataset again to measure any performance increase.

In [23]:
questions = []
with open("/content/drive/MyDrive/Colab Notebooks/openai/eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [24]:
# pass in ft_llm directly into ServiceContext
ft_context = ServiceContext.from_defaults(
    llm=ft_llm,
    context_window=2048,  # limit the context window artifically to test refine process
)

index = VectorStoreIndex.from_documents(docs, service_context=ft_context)

query_engine = index.as_query_engine(similarity_top_k=2)

NameError: name 'ft_llm' is not defined

In [ ]:
contexts = []
answers = []

for question in questions:
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))

In [ ]:
ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

result = evaluate(ds, [faithfulness, answer_relevancy])
print(result)

evaluating with [faithfulness]


100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [07:58<00:00, 239.34s/it]


evaluating with [answer_relevancy]


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]C:\Users\Kishan\anaconda3\Lib\site-packages\tiktoken\core.py:50: RuntimeWarning: coroutine 'DatasetGenerator.agenerate_questions_from_nodes' was never awaited
  self._core_bpe = _tiktoken.CoreBPE(mergeable_ranks, special_tokens, pat_str)
100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:37<00:00, 48.90s/it]


{'ragas_score': 0.9051, 'faithfulness': 0.8375, 'answer_relevancy': 0.9845}


In [ ]:
df_ft_llm = result.to_pandas()

# Export cleaned dataframe as .csv
df_ft_llm.to_csv("eval/df_ft_llm.csv",index=False)

| Pipeline                    | Answer Relevancy | Faithfulness |
|-----------------------------|------------------|--------------|
| RAG+GPT-3.5-Turbo           | -                | -            |
| RAG+Finetuned GPT-3.5-Turbo | 0.9810           | 1.0000       |


Both models are high in performance for both metrics, with the fine-tuned model having an improvement of 1%.
Instead of fine-tuning, we could get comparable performance using the base model, which involves far less computing resource.
And on top of that, as food preferences change over time, RAG alone can easily and quickly adapt to the new data. As such, RAG+GPT-3.5-Turbo is chosen for deployment.

## Exploring Differences

Let's quickly compare the differences in responses, to demonstrate that fine tuning did indeed change something.

In [ ]:
index = VectorStoreIndex.from_documents(docs)

In [ ]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [ ]:
print(questions[1])

### Original

In [ ]:
from llama_index.response.notebook_utils import display_response

query_engine = index.as_query_engine(service_context=gpt_context)

response = query_engine.query(questions[12])

print(response)

### Fine-tuned

In [ ]:
ft_context = ServiceContext.from_defaults(
    llm=ft_llm,
    context_window=2048,  # limit the context window artifically to test refine process
)

In [ ]:
query_engine = index.as_query_engine(service_context=ft_context)

response = query_engine.query(questions[12])

print(response)

**Provide observations here. Sample listed below**

The original base model generated additional sentence "They will provide you with further information on how to fulfill the wishlist items and make your generous contributions." in the answer. This is considered unnecessary information as it is not directly related to the query. The generated answer by the fine-tuned model is concise and sufficiently informative.

[Click for app code in streamlit](../streamlit/restaurantbot.py)